In [1]:
import networkx as nx
import csv
from pymongo import MongoClient
import requests

In [2]:
client = MongoClient()
db = client.recipes

In [24]:
for r in db.elbulli_raw.find():
    if not r['ingredients'] and not r['techniques']:
        print('NO')

In [11]:
ingredients_multidigraph = nx.read_gexf('data/elbulli_ingredients_lexicon_6.gexf')
ingredients_multidigraph = nx.MultiDiGraph(ingredients_multidigraph)
techniques_multidigraph = nx.read_gexf('data/elbulli_techniques_lexicon_6.gexf')
techniques_multidigraph = nx.MultiDiGraph(techniques_multidigraph)

In [4]:
elbulli_ingredientes = set()
elbulli_techniques = set()
for r in db.elbulli_raw.find():
    elbulli_ingredientes = elbulli_ingredientes.union(r['ingredients'])
    elbulli_techniques = elbulli_techniques.union(r['techniques'])

In [5]:
len(elbulli_ingredientes)

1458

In [6]:
len(elbulli_techniques)

221

In [7]:
cookpad_ingredientes = set()
cookpad_techniques = set()
for r in db.cookpad_raw.find():
    cookpad_ingredientes = cookpad_ingredientes.union(r['ingredients'])
    cookpad_techniques = cookpad_techniques.union(r['techniques'])

In [8]:
len(cookpad_ingredientes)

2318

In [9]:
len(cookpad_techniques)

449

In [17]:
len(cookpad_ingredientes.intersection(elbulli_ingredientes))

736

In [18]:
len(cookpad_techniques.intersection(elbulli_techniques))

195

In [19]:
len(cookpad_ingredientes.intersection(ingredients_multidigraph.nodes()))

785

In [20]:
len(cookpad_techniques.intersection(techniques_multidigraph.nodes()))

197

In [31]:
for x in range(10):
    if x %3==0:
        continue
    print(x)

1
2
4
5
7
8
